## 동적 웹페이지 크롤링

- 페이지 오픈 후 자바스크립트 등의 동적인 처리 후 화면이 변경될때는 무조건 동적 웹페이지 크롤링 필요

### selenium
- 기본 라이브러리 pip 설치
- WebDriver 설치
    - 웹브라우저를 제어할 수 있는 컴포넌트 설치 반드시 필요
    - 크롬 웹브라우저 설정 > Chrome정보 현재 버전 확인
    - 그 버전에 맞는 웹드라이버 다운로드 (chromedriver-win64.zip)
    - C:\chromedriver 폴더에 압축파일 내용 해제
    - 윈도우 시스템속성 환경변수 PATH에 크롬드라이버 위치 설정



In [1]:
# 설치
!pip install selenium

In [2]:
from selenium import webdriver

In [8]:
driver = webdriver.Chrome()
driver.get('https://www.naver.com')

#### 스타벅스 매장정보 동적 웹크롤링

- https://www.starbucks.co.kr/store/store_map.do?disp=locale

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By   # bs에서 DOM 접근에 필요한 용어 사용 
from bs4 import BeautifulSoup as bs

import pandas as pd

from tqdm.autonotebook import tqdm # 프로그래스바 시각화 모듈

In [45]:
# 웹드라이버 실행
driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [46]:
## STEP 1, 지역검색
## nth-child(n) : n 번호가 서울, 경기, 광주, 대구... 순
btn_location = 'ul.sido_arae_box > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, btn_location).click()

In [47]:
## STEP 2, 구/군 선택
btn_all = '#mCSB_2_container > ul > li:nth-child(1) > a' ## 전체
driver.find_element(By.CSS_SELECTOR, btn_all).click()

#### BS로 동적으로 변경된 페이지를 정적 크롤링

In [50]:
html = driver.page_source
soup = bs(html, 'html.parser')

In [51]:
len(soup.select('li.quickResultLstCon'))

625

In [52]:
sb_seoul = soup.select('li.quickResultLstCon')

In [53]:
sb_seoul[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [98]:
sb_item = sb_seoul[0]
name = sb_item.select('strong')[0].text.strip() ## 매장이름
lat = sb_item['data-lat'] ## latitude 위도
lng = sb_item['data-long']  ## longitude 경도
address = sb_item.select('p')[0].get_text(separator='|').split('|')[0]
tel = sb_item.select('p')[0].get_text(separator='|').split('|')[1]
type = sb_item.select('i')[0]['class'][0].split('_')[1]

print(name, lat, lng, address, tel, type)

역삼아레나빌딩 37.501087 127.043069 서울특별시 강남구 언주로 425 (역삼동) 1522-3232 general


##### 처리로직 작성

In [99]:
sb_list = []

for sb_item in tqdm(sb_seoul):
    name = sb_item.select('strong')[0].text.strip() ## 매장이름
    lat = sb_item['data-lat'] ## latitude 위도
    lng = sb_item['data-long']  ## longitude 경도
    address = sb_item.select('p')[0].get_text(separator='|').split('|')[0]
    tel = sb_item.select('p')[0].get_text(separator='|').split('|')[1]
    type = sb_item.select('i')[0]['class'][0].split('_')[1]

    sb_list.append([name, lat, lng, address, tel, type])

100%|██████████| 625/625 [00:00<00:00, 3790.79it/s]


In [101]:
len(sb_list)

625

In [102]:
columns = ['매장명', '위도', '경도', '주소', '전화번호', '매장타입']
df_starbucks = pd.DataFrame(data=sb_list, columns=columns)

In [105]:
df_starbucks.tail(2)

,매장명,위도,경도,주소,전화번호,매장타입
623,양원역,37.6066536267232,127.106359790053,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232,general
624,중화역,37.60170912407773,127.07841136432036,서울특별시 중랑구 봉화산로 35 1층,1522-3232,general


In [106]:
df_starbucks.to_csv('./data/스타벅스_서울매장정보.csv', encoding='utf-8', index=False)

### 지도 라이브러리 

- folium

In [107]:
!pip install folium

In [108]:
import folium

In [121]:
# zoom_start 1(지구본)~15(크게)
map = folium.Map(location=[37.550823, 126.989502], tiles='OpenStreetMap', zoom_start=11)

for idx in df_starbucks.index:
    lat = df_starbucks.loc[idx, '위도']
    lng = df_starbucks.loc[idx, '경도']
    name = df_starbucks.loc[idx, '매장명']
    # 우리가 만드는 마커를 지도에 추가
    # folium.Marker(location=[lat, lng], popup=name).add_to(map) 
    folium.CircleMarker(location=[lat, lng], fill=True, fill_color='green', popup=name, color='yellow', weight=2).add_to(map)

map